# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [46]:
import os
import shutil

import azureml.core
from azureml.widgets import RunDetails
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal

SDK version: 1.47.0


## Workspace

In [47]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: quick-starts-ws-219220
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-219220


## Create an Azure ML experiment

Let's create an experiment named "hyperparameter_tuning-experiment" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

In [48]:
experiment_name = 'hyperparameter-tuning-yum'
# project_folder = './hyperparameter-tuning--project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
hyperparameter_tuning-experiment,quick-starts-ws-219220,Link to Azure Machine Learning studio,Link to Documentation


In [49]:
run = experiment.start_logging()

## Create or Attach an AmlCompute cluster

We will need to create a compute target for our Hyperdrive run.

In [50]:
aml_compute_cluster_name = "cluster-yum"

try:
    aml_compute = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print("Found existing cluster, use it.")

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(workspace=ws,
                                       name=aml_compute_cluster_name,
                                       provisioning_configuration=compute_config)
    
aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

In [51]:
# Create an early termination policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        "--C": uniform(0.0001, 1.0),
        "--max_iter": choice(5, 25, 50, 100, 200, 500, 1000)
    }
)

script_folder = "./training"

if "training" not in os.listdir():
    os.mkdir(script_folder)
    
shutil.copy('./train.py', script_folder)

# Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder,
                    compute_target=aml_compute,
                    entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                         hyperparameter_sampling=param_sampling,
                                         primary_metric_name="Accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=40,
                                         max_concurrent_runs=4,
                                         policy=early_termination_policy)

In [52]:
# Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

In the cell below, we use the RunDetails widget to show the different experiments.

In [53]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae
Web View: https://ml.azure.com/runs/HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-219220/workspaces/quick-starts-ws-219220&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-12-19T11:28:10.915336][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-12-19T11:28:11.7436096Z][SCHEDULER][INFO]Scheduling job, id='HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae_0' 
[2022-12-19T11:28:11.8660948Z][SCHEDULER][INFO]Scheduling job, id='HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae_1' 
[2022-12-19T11:28:12.0157815Z][SCHEDULER][INFO]Scheduling job, id='HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae_2' 
[2022-12-19T11:28:12.118594][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-12-19T11:28:12.1593349Z][SCHEDULER][INFO]Scheduling job, id='HD_f36a8c91-66f4-4a85-89f8-5feeb35730

{'runId': 'HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae',
 'target': 'cluster-cpu',
 'status': 'Completed',
 'startTimeUtc': '2022-12-19T11:28:10.274985Z',
 'endTimeUtc': '2022-12-19T12:22:03.245539Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0dda77a5-5069-4e36-a8b9-5861e142eaf8',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': 'infinite_space_size',
  'score': '0.7833333333333333',
  'best_child_run_id': 'HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

## Best Model

In the cell below, we get the best model from the hyperdrive experiments and display all the properties of the model.

In [54]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print("Best Experiment Run:")
print(f" Best Run Id: {best_run.id}")
print(f" Accuracy: {best_run_metrics['Accuracy']}")
# print(f" Regularization Strength: {best_run_metrics['Regularization Strength:']}")
# print(f" Max iterations: {best_run_metrics['Max iterations:']}")

Best Experiment Run:
 Best Run Id: HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae_3
 Accuracy: 0.7833333333333333
 Regularization Strength: 0.6999143187991206
 Max iterations: 500


In [55]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperparameter_tuning-experiment,HD_f36a8c91-66f4-4a85-89f8-5feeb35730ae_3,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


Save the best model

In [57]:
best_model = best_run.register_model(model_name='hyperdrive_model_yum.pkl',
                                     model_path='./outputs/')

best_model.download(target_dir='', exist_ok=True)

'outputs/model.pkl'

In [63]:
joblib.dump(parameter_values, filename='output')

['output']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [65]:
model = Model.register(workspace=ws,model_name = "hyperdrive_model_yum.pkl", model_path = './output')
print(model.name, model.id, model.version, sep='\t')

Registering model best-hyperdrive
best-hyperdrive	best-hyperdrive:1	1


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

